In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


df_train = pd.read_csv('http://bit.ly/kaggletrain')
df_test = pd.read_csv('http://bit.ly/kaggletrain')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,NaN,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,NaN,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,NaN,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,NaN,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,NaN,373450


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Cabin', 'Embarked', 'Name', 'Sex', 'Ticket']



,Cabin,Embarked,Name,Sex,Ticket
0,None,S,"Braund, Mr. Owen Harris",male,A/5 21171
1,C85,C,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599
2,None,S,"Heikkinen, Miss. Laina",female,STON/O2. 3101282
3,C123,S,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803
4,None,S,"Allen, Mr. William Henry",male,373450


觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

均值編碼比較好，但這主要是因為'Name'與'Ticket'兩個欄位的種類數眾多

如果均值編碼時加入這裡個欄位的均值會過擬合( overfitting )

因此下列程式碼中，需要先移除這兩個欄位，再做均值編碼

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.7788902140480825
time : 0.16452288627624512 sec


In [4]:
# 如果欄位類別數太多, 需要先剔除, 否則會 overfit (正確率會到達 1.0)
train_X.nunique()

Cabin       148
Embarked      4
Name        891
Sex           2
Ticket      681
dtype: int64

In [5]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived', 'Name_mean', 'Ticket_mean'] , axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.835019772770071
time : 0.03689908981323242 sec
